In [12]:
#file = open('./data/FullData/fullData.dat','r')

In [8]:
#for line in file:
#    print("{}".format(line.strip()))

In [51]:
file = open('./data/FullData/fullData.dat','r')
line = file.readline()
print(line)
print(line.split('co'))
#print("{}".format(line.strip('co')))
    

# co2a0000364.rd

['# ', '2a0000364.rd\n']


In [52]:
tmp = line.split('co')
group = tmp[1][1]
tmp2 = tmp[1].split('.')
subject = tmp2[0][2:-1]
print('group = ' + group + '\t subject = '+subject)
line = file.readline()
line = file.readline()
line = file.readline()

group = a	 subject = 000036


In [61]:
tmp=line.split(',')[0]
tmp = tmp.split('#')[1] 
print(tmp)

 S1 obj 


In [ ]:
# Opening file
file1 = open('myfile.txt', 'r')
count = 0
 
# Using for loop
print("Using for loop")
for line in file1:
    count += 1
    print("Line{}: {}".format(count, line.strip()))
 
# Closing files
file1.close()